<img src="data/images/lecture-notebook-header.png" />

# Classification & Regression: Logistic Regression

Logistic Regression is a statistical modeling technique used for binary classification tasks, where the goal is to predict the probability of an instance belonging to a certain class. Despite its name, logistic regression is a classification algorithm, not a regression algorithm.

In Logistic Regression, the algorithm models the relationship between the independent variables and the binary outcome using a logistic function, also known as the sigmoid function. The logistic function maps the linear combination of the independent variables to a value between 0 and 1, which represents the probability of belonging to the positive class. This mapping allows logistic regression to estimate the likelihood of an instance belonging to a class and make predictions accordingly.

Although Logistic Regression is called "regression," it is considered a linear model due to its underlying mathematical formulation. The linearity in Logistic Regression refers to the relationship between the independent variables and the log-odds (also known as logit) of the positive class. The log-odds are transformed using the logistic function, which introduces the nonlinearity necessary to model the probability.

The linear part of Logistic Regression comes from the fact that the log-odds of the positive class are expressed as a linear combination of the independent variables. The algorithm determines the coefficients (weights) associated with each independent variable, similar to linear regression. However, instead of predicting the actual continuous value, logistic regression predicts the probability of belonging to the positive class.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Make all Required Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

---

## Working with Toy Data (CSI Example)

As we did in the lecture, we adopt the simple CSI example we used for Linear Regression to a classification task. While the input is still the shoe print size of a person, the output is now a binary class label representing the sex of the person (woman: 0, man: 1).

In [ ]:
data = np.array([
    (31.3, 1), (29.7, 1), (31.3, 0), (31.8, 0),
    (31.4, 1), (31.9, 1), (31.8, 1), (31.0, 1),
    (29.7, 0), (31.4, 1), (32.4, 1), (33.6, 1),
    (30.2, 0), (30.4, 0), (27.6, 0), (31.8, 1),
    (31.3, 1), (34.5, 1), (28.9, 0), (28.2, 0)
])

# Convert input and outputs to numpy arrays; makes some calculations easier
X = data[:,0].reshape(-1,1)
y = data[:,1]

We can still plot the data by using the class label is y coordinate in the scatter plot.

In [ ]:
plt.figure()
plt.tick_params(labelsize=14)
plt.scatter(X, y)
plt.xlabel('Shoe print size (cm)', fontsize=16)
plt.ylabel('P(male)', fontsize=16)
plt.tight_layout()
plt.show()

We can see there is the expected trend that men generally have a larger show print size. But of course, there is no clear separation as there are tall women and small men with corresponding shoe print sizes. That means there will never be a perfect classifier to predict the sex of a person merely based on the size of a shoe print.

### Apply Logistic Regression

scikit-learn provides [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) as implementation for Logistic Regression. Similar to the Linear Regression implementation, the model considers $\theta_0$ (`intercept_`) and $\theta_{i\neq 0}$ (`coef_`) separately. It also features the parameter `fit_intercept` whether to calculate the intercept $\theta_0$ or not.

Below, as we use the original data without adding the constant term ourselves, should set `fit_intercept=True`, which is the default value, so we can simply ignore it.

In [ ]:
clf = LogisticRegression().fit(X, y)

print('Intercept: {}, Coefficients: {}'.format(clf.intercept_, clf.coef_))

We can visualize this result in two ways:

* directly plotting the probabilities (see orange line in the plot below)
* plotting the decision boundary as defined by values for $\theta$

In [ ]:
# Specify series of shoe print size in the range of the input data
x_range = np.arange(27, 35, 0.1).reshape(-1, 1)

# Calculate the probability for all shoe print size
# The method predict_proba() does this for us
y_best = clf.predict_proba(x_range)[:,1]

# Calculate the decision boundary
decision_boundary = clf.intercept_ +  clf.coef_[0] * x_range

We can now plot the probability values and the decision boundary together with the data sample in one figure.

In [ ]:
plt.figure()
plt.ylim(-0.05, 1.05)
plt.tick_params(labelsize=14)
plt.scatter(X, y, c='C0', s=100)
plt.plot(x_range, y_best, color='orange', lw=3)
plt.plot(x_range, decision_boundary, '--', color='black', lw=2)
plt.xlabel('Shoe print size (cm)', fontsize=16)
plt.ylabel('P(male)', fontsize=16)
plt.tight_layout()
plt.show()

As expected, the classification is not perfect.

### Predict Sex of Suspect

In our CSI example, the shoe print size we found of the suspect was 32.2 cm. So take this value as input for our model and look at the prediction. The method `predict()` directly returns the predicted class label (instead of the probabilities).

In [ ]:
y_pred = clf.predict([[32.2]])

print('The predicted class label is: {}'.format(y_pred.squeeze()))

A class label on 1 means the suspect is predicted to be a man. This output can already be seen when looking at the plot above. Also recall that the predicted height for the suspect was 185.7 cm (see notebook for Linear Regression) which is arguably more likely to be a man.

Apart from directly getting the class label, we can also look at the estimated probabilities. This gives us an indication of how "sure" the classifier is about the returned label. Again, we use the method `predict_proba()` for that.

In [ ]:
y_pred = clf.predict_proba([[32.2]])

print('The estimated probabilites are: {}'.format(y_pred.squeeze()))

In the case of our suspect, the difference between the two probabilities is quite large, so we can be reasonably confident that the suspect is indeed a man -- although there will never be a 100% guarantee. Of course, both probabilities add up to 1.

Let's assume the size of the shoe print size of the suspect would have been 30.6 cm. We can estimate the probabilities for this value as well.

In [ ]:
y_pred = clf.predict_proba([[30.6]])

print('The estimated probabilites are: {}'.format(y_pred.squeeze()))

As the probability for Class 1 is still higher than for Class 0, we still would predict the suspect to be a man. However, here the two probabilities are much closer, so we can say the level of confidence of the classifier is much lower. This kind of interpretation is pretty straightforward for binary classification but gets less obvious for multiple classes.

---

## Logistic Regression over Diabetes Dataset

The [Pima Indians Diabetes Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database) is a well-known dataset frequently used in machine learning and data mining research. It contains information about a group of Pima Indian women from Arizona, USA, and their potential risk of developing diabetes. The dataset is commonly used for classification tasks, aiming to predict whether a person has diabetes or not based on various features.

Here are the details of the Pima diabetes dataset:

* Number of instances: 768
* Number of attributes: 8
* Target variable: Outcome (0 for non-diabetic, 1 for diabetic)

The attributes or features in the dataset are:

* Pregnancies: Number of times pregnant.
* Glucose: Plasma glucose concentration after a 2-hour oral glucose tolerance test.
* Blood Pressure: Diastolic blood pressure (mm Hg).
* Skin Thickness: Triceps skinfold thickness (mm).
* Insulin: 2-Hour serum insulin (mu U/ml).
* BMI: Body mass index (weight in kg / (height in m)^2).
* Diabetes Pedigree Function: Diabetes pedigree function, which provides an estimation of the genetic influence.
* Age: Age in years.


### Prepared Training & Test Data

#### Load Dataset from File

In [ ]:
df_diabetes = pd.read_csv('data/datasets/diabetes/diabetes.csv')

# Show the first 5 columns
df_diabetes.head()

#### Generate Training & Test Data

As usual, we convert the dataframe into numpy arrays for further processing, including split the dataset into training and test data.

In [ ]:
# Convert data to numpy arrays
X = df_diabetes[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']].to_numpy()
y = df_diabetes[['Outcome']].to_numpy().squeeze()

# Split dataset in to training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=850)

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))

#### Normalize Data via Standardization

As already mentioned for Linear Regression, normalizing is often not required my in many cases recommended. As the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation applies regularization by default, we do normalize the data via standardization.

In [ ]:
# We fit the scaler based on the training data only
scaler = StandardScaler().fit(X_train)

# Of course, we need to convert both training and test data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Train and Evaluate Logistic Regression Classifier

We directly look into Polynomial Logistic Regression and try different maximum polynomial degrees $p$ (similar to the Linear Regression notebook). Recall from the lecture that the number of terms given a polynomial degree of $p$ a number of input features $d$ is

$$
\#terms = \binom{p+d}{p}
$$

Since our dataset has 8 input features, this equation simplifies to

$$
\#terms = \binom{p+8}{8}
$$

Below we consider $p$ as our hyperparameter, i.e., we transform the dataset using different polynomial degrees, apply Logistic Regression, and check the f1 score for each setup. Note that we no have to set `fit_intercept=False` as  [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) adds the constant term to the data matrix even if $p=1$.

In [ ]:
for p in range(1, 5):
    
    # Transform data w.r.t to degree of polynomial p
    poly = PolynomialFeatures(p)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)
    
    # Train Linear Regressor or transformed data
    # fit_intercept=False since for p=1, transformation adds constant term to data
    poly_reg = LogisticRegression(C=1.0, fit_intercept=False, max_iter=1000).fit(X_train_poly, y_train)

    # Predict values for training and test set
    y_train_pred = poly_reg.predict(X_train_poly)
    y_test_pred = poly_reg.predict(X_test_poly)
    
    # Calculate MSE 
    f1_train = f1_score(y_train, y_train_pred)
    f1_test = f1_score(y_test, y_test_pred)
    
    
    print('Degree of polynomial: {} => f1 (train/test): {:.2f}/{:.2f} (#terms: {})'.format(p, f1_train, f1_test, X_train_poly.shape[1]))

Here basic Logistic Regression without any larger polynomial terms performs best. Increasing $p$ quickly increases the number of polynomial and interaction terms as we have 8 features that allow for many combinations to form the terms. While the training score increases for larger values of $p$, the test score decreases. A clear sign of overfitting.

As [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) not only calculates the predicted class label but also return the respective probabilities, we can also evaluate trained models using the ROC curve and calculate the AU-ROC. For this we can simply re-use the method to plot the ROC curve from a previous notebook.

In [ ]:
def plot_roc_curve(fpr, tpr, roc_auc):
    plt.figure()
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel("False Positive Rate (1-Specificity)", fontsize=16)
    plt.ylabel("True Positive Rate (Sensitivity)", fontsize=16)
    line = plt.plot(fpr, tpr, marker='o', lw=3, markersize=10, label='ROC curve (area = {:.2f})'.format(roc_auc))[0]
    line.set_clip_on(False)
    plt.plot([0, 1], [0, 1], color='black', lw=1, linestyle='--')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()  

The following cell block performs all the required steps of the evaluation:
    
* Transform the data to include polynomial and interaction terms
* Fit a Logistic Regression model
* Calculate the probabilities in the test set
* Plot the ROC curve with the AUC-ROC value

You can try different values of $p$ and see how it changes the plot.

In [ ]:
poly = PolynomialFeatures(1)
#poly = PolynomialFeatures(2)
#poly = PolynomialFeatures(3)
#poly = PolynomialFeatures(4)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)
    
# Train Linear Regressor or transformed data
# fit_intercept=False since for p=1, transformation adds constant term to data
poly_reg = LogisticRegression(C=1.0, fit_intercept=False, max_iter=1000).fit(X_train_poly, y_train)

# Predict values and probabilities for test set
y_pred = poly_reg.predict(X_test_poly)
y_pred_proba = poly_reg.predict_proba(X_test_poly)

# As we have a binary task, we can focus on the probability for Class 1.
y_pred_proba = y_pred_proba[:,1]
    
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
plot_roc_curve(fpr, tpr, roc_auc)

Lastly, we can also perform a more proper evaluation using k-fold cross-validation to find the best value of $p$. To simplify things, we use the scikit-learn's method `cross_val_score()` to perform the cross-validation for us.

In [ ]:
# Initialize the best f1-score and respective k value
best_p, best_f1 = None, 0.0

# Loop over a range of values for setting k
for p in range(1, 4):
    
    # Transform data w.r.t to degree of polynomial p
    poly = PolynomialFeatures(p)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)
    
    # Specfify type of classifier
    clf = LogisticRegression(fit_intercept=False, max_iter=1000)
    
    # perform cross validation (here with 10 folds)
    # f1_scores is an array containg the 10 f1-scores
    f1_scores = cross_val_score(clf, X_train_poly, y_train, cv=10)
    
    # Calculate the f1-score for the current k value as the mean over all 10 f1-scores
    f1_fold_mean = np.mean(f1_scores)

    print('p={}, f1 score (mean/std): {:.3f}/{:.3f}'.format(p, f1_fold_mean, np.std(f1_scores)))
    
    # Keep track of the best f1-score and the respective k value
    if f1_fold_mean > best_f1:
        best_p, best_f1 = p, f1_fold_mean
  

print('The best f1-score was {:.3f} for p={}'.format(best_f1, best_p))

Again, $p=1$ yields the best average f1 score although the difference compared to the results for $p=2$ is rather small.

Having found the best value for $p$ we can now fit a Logistic Regression model on the whole training data and $p=1$ and evaluate the f1 score on the test data.

In [ ]:
# Transform data
poly = PolynomialFeatures(1)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

# Fite Logistic Regression model on complete training data
clf = LogisticRegression(fit_intercept=False, max_iter=1000).fit(X_train, y_train)

# Predict class labels for test data
y_pred = clf.predict(X_test)

# Calculate f1 scores based on ground truth of test set
f1 = f1_score(y_test, y_pred)

print('F1 score of Linear Regression model on the test data: {:.3f}'.format(f1))

---

## Summary

Logistic Regression is a popular machine learning algorithm used for binary classification tasks, where the goal is to predict the probability of an instance belonging to a certain class. Despite its name, logistic regression is a classification algorithm, not a regression algorithm. It is called "Logistic Regression" because it is based on the concept of logistic function or sigmoid function.

In Logistic Regression, the algorithm models the relationship between the independent variables and the binary outcome using a logistic function. The logistic function maps the linear combination of the independent variables to a value between 0 and 1, which represents the probability of belonging to the positive class. This mapping allows logistic regression to estimate the likelihood of an instance belonging to a class and make predictions accordingly.

One of the key advantages of Logistic Regression is its simplicity and interpretability. The algorithm provides coefficients for each independent variable, allowing us to understand the impact and direction of each variable on the probability of the positive class. However, logistic regression has some limitations. It assumes a linear relationship between the independent variables and the log-odds of the positive class, which may not always hold true. It may struggle with nonlinear relationships or complex interactions between variables. Additionally, logistic regression is sensitive to outliers and can be affected by overfitting when the number of independent variables is large compared to the number of instances.

In summary, logistic regression is a straightforward and interpretable algorithm for binary classification tasks. Its pros include simplicity, interpretability, and handling of categorical and continuous variables. However, it has limitations in capturing nonlinear relationships and can be sensitive to outliers and overfitting. Therefore, it is important to assess the assumptions and limitations of logistic regression before applying it to a given problem.